In [1]:
import joblib
data = joblib.load("boxscore_2025.pkl")

In [2]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import numpy as np

# xgf = 60 * (data["stints"]["xgf"] / data["stints"]["duration"])

# fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# sns.histplot(xgf, bins=100, ax=axes[0])
# axes[0].set_title('All xGF values')

# sns.histplot(xgf[xgf > 0], bins=100, ax=axes[1])
# axes[1].set_title('xGF > 0')

# sns.histplot(np.log(xgf[xgf > 0]), bins=100, ax=axes[2])
# axes[2].set_title('Log(xGF) where xGF > 0')

# plt.tight_layout()
# plt.show()

In [3]:
import nhl_scraper.model as m

def get_rapm(stints, boxscore, season=None):
  if season is not None:
    stints = stints[stints["season"] == season]
    boxscore = boxscore[boxscore["season"] == season]


  xgf = m.hurdle_model(boxscore, 
                              stints, 
                              metric="xgf", 
                              occ_metric="ff",
                              cv=2, 
                              n_alphas=10)

  xga = m.hurdle_model(boxscore, 
                              stints, 
                              metric="xga", 
                              occ_metric="fa",
                              cv=2, 
                              n_alphas=10)




  player_impact = xga.merge(xgf, on=["playerId", "name", "position", "teamId"])
  player_impact = player_impact.drop_duplicates(subset=["playerId"])
  if season is not None:
    player_impact["season"] = season
  return player_impact

In [4]:
coef_2025 = get_rapm(data["stints"], data["boxscore"]) 


fitting logistic regression grid search
Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END ..............................................C=1.0; total time=   7.7s
[CV] END ..............................................C=1.0; total time=   7.4s
[CV] END ...............................C=0.5013193223677261; total time=   8.2s
[CV] END ...............................C=0.5013193223677261; total time=   8.8s
[CV] END ...............................C=0.2513210629792362; total time=   8.9s
[CV] END ...............................C=0.2513210629792362; total time=   8.4s
[CV] END ..............................C=0.12599210498948735; total time=   8.3s
[CV] END ..............................C=0.12599210498948735; total time=   8.4s
[CV] END ...............................C=0.0631622766970132; total time=   8.3s
[CV] END ...............................C=0.0631622766970132; total time=   8.3s
[CV] END .............................C=0.031664469752949474; total time=   8.3s
[CV] END

In [5]:
import joblib
import pandas as pd
import numpy as np
import gc

old = joblib.load("boxscore.pkl")
stints = old["stints"]

stints["season"] = stints["game"].astype(str).str[:4]

cols = stints.columns.tolist()
batch = 100

for i in range(0, len(cols), batch):
    sub = cols[i:i+batch]
    float_cols = stints[sub].select_dtypes("float64").columns
    int_cols   = stints[sub].select_dtypes("int64").columns

    for col in float_cols:
        stints[col] = pd.to_numeric(stints[col], downcast="float")
    for col in int_cols:
        stints[col] = pd.to_numeric(stints[col], downcast="integer")

    gc.collect()

null_cols = [c for c in stints.columns if stints[c].isnull().any()]
batch = 100

for i in range(0, len(null_cols), batch):
    sub = null_cols[i:i+batch]
    for col in sub:
        arr = stints[col].to_numpy()
        if arr.dtype.kind == "f":
            mask = np.isnan(arr)
            if mask.any():
                arr[mask] = 0
        else:
            stints[col] = stints[col].fillna(0)
    gc.collect()

del old
gc.collect()

boxscore = joblib.load("boxscore.pkl")["boxscore"]
boxscore["season"] = boxscore["game"].astype(str).str[:4]


/var/folders/7y/fwm3j5vs3p72s234bsqbz8sw0000gp/T/ipykernel_8852/2068579207.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stints["season"] = stints["game"].astype(str).str[:4]


In [6]:
import pandas as pd

coef = coef_2025

for season in ["2021", "2022", "2023", "2024"]:
    coef = pd.concat([get_rapm(stints, boxscore, season), coef])
    coef = coef.sort_values("season")
    coef.to_csv("decomp_rapm.csv")



/Users/K-Dog/Desktop/Projects/nhl-scraper/nhl_scraper/model.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["manpower"] = df["manpower"].astype("category")


fitting logistic regression grid search
Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END ..............................................C=1.0; total time=  10.6s
[CV] END ..............................................C=1.0; total time=  10.6s
[CV] END ...............................C=0.5013193223677261; total time=   9.1s
[CV] END ...............................C=0.5013193223677261; total time=   9.9s
[CV] END ...............................C=0.2513210629792362; total time=   9.2s
[CV] END ...............................C=0.2513210629792362; total time=   9.9s
[CV] END ..............................C=0.12599210498948735; total time=   9.6s
[CV] END ..............................C=0.12599210498948735; total time=   8.9s
[CV] END ...............................C=0.0631622766970132; total time=   8.0s
[CV] END ...............................C=0.0631622766970132; total time=   8.2s
[CV] END .............................C=0.031664469752949474; total time=   8.7s
[CV] END

/Users/K-Dog/Desktop/Projects/nhl-scraper/nhl_scraper/model.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["manpower"] = df["manpower"].astype("category")


fitting logistic regression grid search
Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END ..............................................C=1.0; total time=  10.8s
[CV] END ..............................................C=1.0; total time=  11.1s
[CV] END ...............................C=0.5013193223677261; total time=   9.3s
[CV] END ...............................C=0.5013193223677261; total time=   9.7s
[CV] END ...............................C=0.2513210629792362; total time=   9.0s
[CV] END ...............................C=0.2513210629792362; total time=   9.7s
[CV] END ..............................C=0.12599210498948735; total time=   8.4s
[CV] END ..............................C=0.12599210498948735; total time=   9.0s
[CV] END ...............................C=0.0631622766970132; total time=   8.1s
[CV] END ...............................C=0.0631622766970132; total time=   8.3s
[CV] END .............................C=0.031664469752949474; total time=   8.7s
[CV] END

/Users/K-Dog/Desktop/Projects/nhl-scraper/nhl_scraper/model.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["manpower"] = df["manpower"].astype("category")


...

fitting logistic regression grid search
Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END ..............................................C=1.0; total time=   9.7s
[CV] END ..............................................C=1.0; total time=   9.2s
[CV] END ...............................C=0.5013193223677261; total time=   9.0s
[CV] END ...............................C=0.5013193223677261; total time=   8.2s
[CV] END ...............................C=0.2513210629792362; total time=   9.5s
[CV] END ...............................C=0.2513210629792362; total time=   8.8s
[CV] END ..............................C=0.12599210498948735; total time=   8.6s
[CV] END ..............................C=0.12599210498948735; total time=   8.8s
[CV] END ...............................C=0.0631622766970132; total time=   9.0s
[CV] END ...............................C=0.0631622766970132; total time=   9.0s
[CV] END .............................C=0.031664469752949474; total time=   8.3s
[CV] END

/Users/K-Dog/Desktop/Projects/nhl-scraper/nhl_scraper/model.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["manpower"] = df["manpower"].astype("category")


fitting logistic regression grid search
Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END ..............................................C=1.0; total time=   9.6s
[CV] END ..............................................C=1.0; total time=   9.7s
[CV] END ...............................C=0.5013193223677261; total time=   8.7s
[CV] END ...............................C=0.5013193223677261; total time=   7.8s
[CV] END ...............................C=0.2513210629792362; total time=   8.7s
[CV] END ...............................C=0.2513210629792362; total time=   8.5s
[CV] END ..............................C=0.12599210498948735; total time=   8.3s
[CV] END ..............................C=0.12599210498948735; total time=   7.9s
[CV] END ...............................C=0.0631622766970132; total time=   8.1s
[CV] END ...............................C=0.0631622766970132; total time=   8.6s
[CV] END .............................C=0.031664469752949474; total time=   8.2s
[CV] END

/Users/K-Dog/Desktop/Projects/nhl-scraper/nhl_scraper/model.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["manpower"] = df["manpower"].astype("category")


fitting logistic regression grid search
Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END ..............................................C=1.0; total time=  10.3s
[CV] END ..............................................C=1.0; total time=   8.7s
[CV] END ...............................C=0.5013193223677261; total time=   9.9s
[CV] END ...............................C=0.5013193223677261; total time=   9.8s
[CV] END ...............................C=0.2513210629792362; total time=   9.1s
[CV] END ...............................C=0.2513210629792362; total time=   8.6s
[CV] END ..............................C=0.12599210498948735; total time=   7.7s
[CV] END ..............................C=0.12599210498948735; total time=   7.8s
[CV] END ...............................C=0.0631622766970132; total time=   7.0s
[CV] END ...............................C=0.0631622766970132; total time=   7.9s
[CV] END .............................C=0.031664469752949474; total time=   7.6s
[CV] END

In [9]:
coef.fillna("2025").to_csv("decomp_rapm.csv")

In [7]:
rank_df = coef.groupby(by=["position", "season"])